# 기상 데이터 전처리

## 기상 데이터 전처리 (강수확률)


제주도 지역별 강수확률 데이터를 가져온 후, 조건에 맞는 값을 예측값으로 지정하여 확인한 후 최종 강수확률 데이터로 저장

[데이터셋 변수 설명] 
- 일 : 일자 (1~31)
- UTC : 예측 시간(그리니치 천문대 기준 -> 실제 한국 시간으로는 +9시간)
- 예측 + 시간 : UTC 변환 이후 몇시간 뒤의 데이터를 예측할지 (EX, 7이면 7시간 이후)
- 강수 확률 : 강수확률 (단위 : %)

[전처리 과정 설명] 
2:00(UTC시간대 기준)의 7시간 뒤의 예측값만을 추출하여 데이터프레임으로 구성하고자 함 
- UTC == 200, 예측 + 시간 == 7인 행만을 조건 아래 추출 
- 조건 색인 이후 불필요한 열인 UTC , 예측+시간 컬럼을 drop
- 1~30의 int 형태로 나열되어 있는 일 컬럼을 날짜 형태(2021-01-01 ~ 2021-12-31)로 변환 
    
[기타 데이터 관련 설명] 
- 제주시 및 서귀포시 기준 행정동은 네이버 날씨의 대표값을 따름 (천지동의 경우 네이버 날씨에서는 서귀동을 기준으로 하지만 냘씨 예보는 법정동을 기준으로 하여 그에 준응하는 천지동의 데이터를 사용함) 
- 각 일자에 대한 예측값은 UTC기준 2:00AM (한국 시간, 11:00AM)에 발표된 해당 날짜의 18:00(KST)에 대한 예측값을 사용함

source : 기상청 - 기상자료개방포털 (2021/01-2021/06은 (구)단기 예보 파일셋 활용, 2021/07-2021/12은 단기 예보 파일셋 활용)
https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572

In [54]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

In [55]:
# 제주시, 서귀포시 데이터 불러오기
df1 = pd.read_csv('../Dataset/raw_data/weather/precip_prob/precip_prob_j/아라동_강수확률_202101_202106.csv') # 제주시 전반기
df2 = pd.read_csv('../Dataset/raw_data/weather/precip_prob/precip_prob_j/아라동_강수확률_20210701_20211231.csv') # 제주시 후반기
# df2 = pd.read_csv('아라동_강수확률_20210701_20211231.csv')
df3 = pd.read_csv('../Dataset/raw_data/weather/precip_prob/precip_prob_s/천지동_강수확률_202101_202106.csv') # 서귀포시 전반기
df4 = pd.read_csv('../Dataset/raw_data/weather/precip_prob/precip_prob_s/천지동_강수확률_20210701_20211231.csv', dtype=str) # 서귀포시 후반기
# df4 = pd.read_csv('천지동_강수확률_20210701_20211231.csv', dtype=str) # 서귀포시 후반기

In [56]:
df1.shape, df2.shape, df3.shape, df4.shape

((26793, 4), (93656, 4), (26793, 4), (93656, 4))

In [57]:
# 모든 데이터 칼럼명 재지정
df1.columns = ['일','UTC','예측+시간','강수확률']
df2.columns = ['일','UTC','예측+시간','강수확률']
df3.columns = ['일','UTC','예측+시간','강수확률']
df4.columns = ['일','UTC','예측+시간','강수확률']

In [58]:
df4

,일,UTC,예측+시간,강수확률
0,1,200,6,0
1,1,200,7,20
2,1,200,8,0
3,1,200,9,0
4,1,200,10,0
...,...,...,...,...
93651,31,2300,61,0
93652,31,2300,62,0
93653,31,2300,63,0
93654,31,2300,64,0


In [59]:
# 특정 조건에 해당하는 칼럼만 데이터로 지정
# 조건 : UTC->200, 예측+시간->7.0 (그리니치 천문대 기준 오전 2시, 즉 한국 기준 11시에서 예측한 7시간 뒤의 기온 -> 오후 6시 기온)
fc1 = df1[(df1['UTC']==200.0) & (df1['예측+시간']==7.0)] # 아라동 전반기
fc2 = df2[(df2['UTC']==200.0) & (df2['예측+시간']==7.0)] # 아라동 후반기
fc3 = df3[(df3['UTC']==200.0) & (df3['예측+시간']==7.0)] # 천지동 전반기
fc4 = df4[(df4['UTC']=='200') & (df4['예측+시간']=='7')] # 천지동 후반기

In [60]:
fc1.shape, fc2.shape, fc3.shape, fc4.shape

((181, 4), (184, 4), (181, 4), (184, 4))

In [61]:
# 함수를 이용해서 2021-01-01부터 2021-12-31까지 모든 데이터 불러옴
from datetime import datetime, timedelta
# 해당 범위 내의 날짜들을 리스트로 지정해주는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates
# 2021-01-01 ~ 2021-12-31 사이의 날짜를 리스트로 지정
date_list = date_range("2021-01-01", "2021-12-31")
# 대표적으로 앞의 5개만 추출
date_list[:5]

['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05']

In [62]:
# 제주시 데이터를 합침
ara = pd.concat([fc1,fc2])
# date_list를 ara데이터의 날짜 칼럼에 저장
ara['날짜'] = date_list
# 순서 지정 및 특정 데이터만 가져옴
ara = ara[['날짜','일','강수확률']]
# 길이 확인(1월1일부터 12월 31일까지 이상없이 있는지)
len(ara)

365

In [63]:
# 서귀포시 데이터를 합침
cheonji = pd.concat([fc3,fc4])
# date_list를 cheonji데이터의 날짜 칼럼에 저장
cheonji['날짜'] = date_list
# 순서 지정 및 특정 데이터만 가져옴
cheonji = cheonji[['날짜','일','강수확률']]
# 길이 확인(1월1일부터 12월 31일까지 이상없이 있는지)
len(cheonji)

365

In [172]:
# 데이터 저장
ara.to_csv('아라동강수확률.csv',index=False, encoding = 'utf-8-sig')
cheonji.to_csv('천지동강수확률.csv',index=False, encoding = 'utf-8-sig')

In [189]:
# 일 데이터가 이상이 없는지 확인(ex. 1일이 12개, 30일이 11개..)
cheonji['일'].value_counts()

 1     12
 2     12
 28    12
 27    12
 26    12
 25    12
 24    12
 23    12
 22    12
 21    12
 20    12
 19    12
 18    12
 17    12
 16    12
 15    12
 14    12
 13    12
 12    12
 11    12
 10    12
 9     12
 8     12
 7     12
 6     12
 5     12
 4     12
 3     12
 29    11
 30    11
 31     7
Name: 일, dtype: int64